#### 1. Notebook introduction

>In this notebook you will find ou will find the functions to generate the different types of maps.To execute the function, you must include the files upz-cods.csv and lcl-cods.csv in your local path.

> You must supply the **generate_chorop_geojson** function, the following list of arguments:

>* ***file_path:*** local path where you have you have your csv or xlsx files that contain the information you want to convert into geojson.  Additionally, you must include the above-mentioned **lcl-cods.csv** and **upz-cods.csv** files.
> * ***prop_file:*** name of the file (with extension) that contains the set of properties that you want to be generated in the geojson file. The properties file must contain the unit code for each record in the same file in a column. The code will depend on the spatial unit used. The column code for each spatial unit is indicated below.
>> * column name for UPZ: COD_UPZ
>> * column name for localities: COD_LCL
>> * column name for cadastral zones: COD_ZC
> * ***sp_unit:*** spatial unit for which the file will be generated. You must enter as an argument any of the following spatial units:
>> * UPZ = upz
>> * localidades = lcl
>> * zona catrastral = zc
> * **init_scale and end_scale:** these two variables correspond to the maximum and minimum zoom level at which the data will be displayed as a layer within the visualization system. For more information you can explore https://docs.mapbox.com/help/glossary/zoom-level/ and validate the scale. The minimum allowed value is 0 and the maximum allowed is 22.
> * ***c_palete:*** suggested color palette for data going to choropleth maps. You should select any of the items found in the array. If you do not know the colors associated with hexadecimal codes, you can see them at the following link https://htmlcolorcodes.com/. These selected values will be the ones with which the display system will generate the color scale. You must select the index of the palette you want to use.
> * ***qcolors:*** number of intermediate colors in the selected palette.
> * ***title:*** short text that describes the context of the data. Example, Monthly Incidents, Crime Prediction Factors.
> * ***file_name:*** name of the file that will be exported in geojson format (it is not necessary to specify the extension).

In [3]:
# function to create a gejson choropleth map
import pandas as pd
import numpy as np
import geojson
from geojson import FeatureCollection, Feature, Point 
import json

color_palete = [['#EDE907', '#ED2D07'],
                ['#05A6F7', '#05F726'],
                ['#FFFF00', '#00FF00'],
                ['#F1C40F', '#D35400'],
                ['#BDC3C7', '#2C3E50']]

def generate_chorop_geojson (file_path, prop_file, sp_unit, init_scale = 10, 
                             end_scale = 18, c_palete = 3, qcolors = 4, lts_color_scale='q', 
                             categorical=False, title='N/A', file_name='chorop data'):
    
    # path file dataframe
    cods_file = ''
    # dataframe to lons and lats for spatial units
    df_untis = pd.DataFrame()
    # list of properties 
    df_prop = pd.DataFrame()
    # meta data geojson file
    
    meta_data = {}
    cat = 0
    color_intervals = lts_color_scale
    color_palete = [['#EDE907', '#ED2D07'],
                    ['#05A6F7', '#05F726'],
                    ['#FFFF00', '#00FF00'],
                    ['#F1C40F', '#D35400'],
                    ['#BDC3C7', '#2C3E50']]
    match_col = ''
    if sp_unit == 'upz':
        match_col = 'COD_UPZ'
        cods_file = 'upz-cods.csv'
    elif sp_unit == 'lcl':
        match_col = 'COD_LCL'
        cods_file = 'lcl-cods.csv'
    elif sp_unit == 'zc':
        match_col = 'COD_ZC'
        cods_file = 'zc-cods.csv'

    # reading the unit centroid file and the properties file
    df_units = pd.read_csv(file_path + cods_file, encoding='utf-8',
                           error_bad_lines=False)

    # validating the file type and reading
    if prop_file.endswith('.csv') or prop_file.endswith('.CSV'):
        df_prop = pd.read_csv(file_path + prop_file, encoding='utf-8', 
                                error_bad_lines=False)
    if prop_file.endswith('.xlsx') or prop_file.endswith('.XLSX'):
        df_prop = pd.read_excel(file_path + prop_file, encoding='utf-8', 
                                error_bad_lines=False) 
    
    
    if (len(df_prop) > 0 and (init_scale >= 0 and init_scale <= 20) and 
        (end_scale >= 0 and end_scale <= 20) and (c_palete >= 0 and c_palete <= 5)):
        
        lst_features = list()           
        size = len(df_prop)
        
        for i in range(size):
            prop = {}
            
            # find the lon and lat for one property
            unit_cod = df_prop[match_col].iloc[i]
            rec = df_units[df_units[match_col].isin([unit_cod])]
            lon = float(rec['LONS'].iloc[0])
            lat = float(rec['LATS'].iloc[0])
            
            point = Point((lon, lat))
            for j in range (len(df_prop.columns)):
                
                val = df_prop[df_prop.columns[j]].iloc[i] 
                if match_col == 'COD_LCL' and df_prop.columns[j] == 'COD_LCL':
                    val = df_prop[df_prop.columns[j]].iloc[i]
                    if type(val) is np.int64:
                        val = str(val)
                    if len(val) == 1:
                        val = '0'+ str(val)
                    else:
                        val = str(df_prop[df_prop.columns[j]].iloc[i])
                    
                if type(val) is np.int64:
                    val = int(val)
                if type(val) is np.float64:
                    val = float(val)
                prop[df_prop.columns[j]] = val

            feature = Feature(geometry = point, id = i, properties = prop)
            lst_features.append(feature)
            # print(prop)
        
        if categorical:
            cat = 1
            color_intervals = 'e'
            
            
        meta_data = {
            'init_scale': init_scale,
            'end_scale': end_scale,
            'c_palete': color_palete[c_palete],
            'qcolors': qcolors,
            'title': title,
            'cat': cat,
            'type_scale': color_intervals
        }

        fc_geojson = FeatureCollection(lst_features)
        fc_geojson['config'] = meta_data
        #print(fc_geojson)
        f = open(file_path + file_name.strip() + '.geojson', 'w')
        f.write(str(fc_geojson))
        f.close()
        print('geojson generated....................')
        return fc_geojson
    else:
        print('Input data error. Validate:  \n',
             ' 1. Scale values must be between 0 and 20 \n',
             ' 2. Color palette must be between 0 and 4')
    

#### 2.1 Function invocation example (generate_chorop_geojson)
> The following cell shows an example of how to invoke the function. The function sets some default values ​​in case you don't want to specify them. In this example, the scale values, color palette, and number of colors are not being entered as arguments. For the creation of the file, the default values ​​that the function has will be assigned.

In [8]:
geofile = generate_chorop_geojson (file_path='',
                                   prop_file='grupos-categoricos2.csv',
                                   sp_unit='upz',
                                   title='Categorias X',
                                   categorical=True,
                                   qcolors = 4,
                                   file_name='catgrupos2')

geojson generated....................
